In [1]:
import pytfa

from optlang.exceptions import SolverError

from cobra.core.model import SolverNotFound
from cobra.flux_analysis import flux_variability_analysis
import cobra


from pytfa.io import import_matlab_model, load_thermoDB,                    \
                            read_lexicon, annotate_from_lexicon,            \
                            read_compartment_data, apply_compartment_data


Academic license - for non-commercial use only - expires 2021-06-15
Using license file /home/mdz/gurobi.lic


In [2]:
# load model with growth on GG and check solver. Set to gurobi if necessary
cobra_config = cobra.Configuration()
cobra_config.solver = "gurobi"
model = cobra.io.read_sbml_model('v7_tac125_gg_growth_bounds.xml')
model.solver

In [3]:
# adjust model to pyTFA requirements
# bring all max reaction bounds to max == +/- 1000
for reaction in model.reactions:
    if reaction.upper_bound > 1000:
        reaction.upper_bound = 1000
    if reaction.lower_bound < -1000:
        reaction.lower_bound = -1000

In [5]:
# Load reaction DB
print("Loading thermo data...")

thermo_data = load_thermoDB('pytfa_data/thermo_data.thermodb')
print("Done !")

Loading thermo data...
Done !


In [6]:
lexicon = read_lexicon('tac125_lexicon.csv')
compartment_data = read_compartment_data('tac125_compartment_data.json')

# Initialize the cobra_model
mytfa = pytfa.ThermoModel(thermo_data, model)

# Annotate the cobra_model
annotate_from_lexicon(mytfa, lexicon)
apply_compartment_data(mytfa, compartment_data)


Read LP format model from file /tmp/tmp_fiu1hek.lp
Reading time = 0.01 seconds
: 1136 rows, 2654 columns, 11540 nonzeros


2021-04-28 08:04:33,190 - thermomodel_ - INFO - # Model initialized with units kcal/mol and temperature 298.15 K


In [9]:
solver = 'gurobi'
biomass_rxn = 'RXNbiomass'
mytfa.name = 'tutorial_basics'
mytfa.solver = solver
mytfa.objective = biomass_rxn

In [10]:
# Solver settings
def apply_solver_settings(model, solver = solver):
    model.solver = solver
    # model.solver.configuration.verbosity = 1
    model.solver.configuration.tolerances.feasibility = 1e-9
    if solver == 'gurobi':
        model.solver.problem.Params.NumericFocus = 3
    model.solver.configuration.presolve = True

apply_solver_settings(mytfa)

In [11]:
## FBA
fba_solution = model.optimize()
fba_value = fba_solution.objective_value
fba_value
# expected ~ 1.1

1.0703446702156532

In [12]:
## TFA conversion
mytfa.prepare()
mytfa.convert()#add_displacement = True)

## Info on the cobra_model
#mytfa.print_info()

2021-04-28 08:06:13,848 - thermomodel_ - INFO - # Model preparation starting...
2021-04-28 08:06:14,950 - thermomodel_ - WARNING - Warning : Pb/Pb
2021-04-28 08:06:14,951 - thermomodel_ - WARNING - Warning : Pb/Pb
2021-04-28 08:06:15,066 - thermomodel_ - INFO - # Model preparation done.
2021-04-28 08:06:15,067 - thermomodel_ - INFO - # Model conversion starting...


KeyError: <Metabolite cpd15672_c at 0x7f19c7a62e10>